In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": 0},
    attn_implementation="sdpa",
)
# model = model.eval()
model.requires_grad_(False)

from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    # "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    # test_size=250,
    test_size=350,
    # test_size=999,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)

correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

print(len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset))

correct_dataset = correct_dataset[:30]
len(correct_dataset)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
224 202 0.9017857142857143


30

### Default generation from embeds

In [3]:
import torch

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm

correct_items = 0
torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()
# tokenizer = AutoTokenizer.from_pretrained(model_name)

evaluation_dataset = []
# dataset_pos = 0
# for dataset_pos in tqdm(range(len(correct_dataset))):
dataset_pos = 5
input_ids = correct_dataset[dataset_pos]["problem"]
# print(input_ids)
# print("===")
# print("===")

input_ids = [
    tokenizer.apply_chat_template(
        [
            {
                "role": "user",
                "content": base_prompt.format(question=input_ids),
            },
        ],
        tokenize=True,
        add_generation_prompt=True,
    )
]

# input_ids = dataset_item["input_ids"]
# generated_ids = dataset_item["generated_ids"]
generated_ids = [
    tokenizer.encode(
        correct_dataset[dataset_pos]["model_answer"],
        add_special_tokens=False,
    )
]

device = "cuda"

with torch.no_grad():

    input_ids = torch.tensor(input_ids).cuda()
    input_ids_embeds = model.get_input_embeddings()(input_ids)
    windows_amount = 100

    generated_embeds = torch.cat(
        [
            input_ids_embeds,
        ],
        dim=1,
    )
    # print("COMPRESSED PART", tokenizer.decode(next_true_tokens[-1]))
    # print("===")
    generated_ids_compressed = model.generate(
        inputs_embeds=generated_embeds,
        # attention_mask=torch.ones_like(generated_embeds),
        attention_mask=torch.ones(
            generated_embeds.shape[:2],
            device="cuda",
        ).long(),
        max_new_tokens=4096,
        # max_new_tokens=5,
        do_sample=False,
        # do_sample=True,
        # temperature=0.6,
        # top_p=0.95,
    )
    # break
generated_result = tokenizer.decode(generated_ids_compressed[-1])
# print()
gold_answer = correct_dataset[dataset_pos]["answer"]
answer = dataset_answer_filter(gold_answer)
model_answer = model_answer_filter(generated_result)
if is_equiv(answer, model_answer):
    correct_items += 1
    print("CORRECT")
else:
    print("WRONG", gold_answer)
    print(generated_result)
compressed_total_len = generated_ids_compressed.shape[1]
print(
    generated_ids_compressed.shape[1],
    compressed_total_len,
    torch.tensor(generated_ids).shape[1],
)

/home/user-name-goes-here/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/user-name-goes-here/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


CORRECT
1081 1081 1080


## test time train generation

In [4]:
correct_dataset[dataset_pos]

{'problem': 'In right triangle $ABC$ with $\\angle B = 90^\\circ$, we have $\\sin A = 2\\cos A$.  What is $\\tan A$?',
 'solution': 'The triangle is shown below:\n\n[asy]\npair A,B,C;\nA = (0,0);\nB = (5,0);\nC = (5,10);\ndraw(A--B--C--A);\ndraw(rightanglemark(C,B,A,16));\nlabel("$A$",A,SW);\nlabel("$B$",B,SE);\nlabel("$C$",C,N);\n[/asy]\n\nWe have $\\sin A = \\frac{BC}{AC}$ and $\\cos A = \\frac{AB}{AC}$, so $\\sin A = 2\\cos A$ gives us $\\frac{BC}{AC} = 2\\cdot\\frac{AB}{AC}$.  Multiplying both sides by $AC$ gives $BC = 2AB$, so $\\frac{BC}{AB} = 2$.  Finally, we have $\\tan A = \\frac{BC}{AB} = \\boxed{2}$.\n\nWe also could have noted that $\\tan A = \\frac{\\sin A}{\\cos A} = \\frac{2\\cos A}{\\cos A } =\\boxed{2}$.',
 'answer': '2',
 'subject': 'Geometry',
 'level': 3,
 'unique_id': 'test/geometry/178.json',
 'model_answer': "Okay, so I have this problem here: In right triangle ABC with angle B equal to 90 degrees, we know that sin A equals 2 cos A. The question is asking for tan

In [17]:
import torch

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm
from hidden_capacity_reasoning.utils import tokenize_single_turn

correct_items = 0
torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

# dataset_pos = 5
dataset_pos = 6
tokenized_turn = tokenize_single_turn(
    question=base_prompt.format(question=correct_dataset[dataset_pos]["problem"]),
    answer=correct_dataset[dataset_pos]["model_answer"],
    tokenizer=tokenizer,
)
for key in tokenized_turn.keys():
    tokenized_turn[key] = torch.tensor(tokenized_turn[key])

evaluation_dataset = []


input_ids = correct_dataset[dataset_pos]["problem"]

input_ids = [
    tokenizer.apply_chat_template(
        [
            {
                "role": "user",
                "content": base_prompt.format(question=input_ids),
            },
        ],
        tokenize=True,
        add_generation_prompt=True,
    )
]

device = "cuda"


content_compression_mask = torch.tensor(tokenized_turn["content_compression_mask"])

input_part_end = (content_compression_mask == 0).nonzero()[-3][0]
question_input_ids = tokenized_turn["input_ids"][: int(input_part_end) + 1].unsqueeze(0)
print(tokenizer.decode(question_input_ids[-1]))

with torch.no_grad():

    input_ids = torch.tensor(input_ids).cuda()
    input_ids_embeds = model.get_input_embeddings()(input_ids)
    max_new_tokens = 400

    inputs_embeds = torch.cat(
        [
            input_ids_embeds,
        ],
        dim=1,
    )
    generated_ids_new = model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=torch.ones(
            inputs_embeds.shape[:2],
            device="cuda",
        ).long(),
        max_new_tokens=max_new_tokens,
        do_sample=False,
    )
    # break
generated_result = tokenizer.decode(generated_ids_new[-1])
print(generated_result)

/tmp/ipykernel_1197467/409245293.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  content_compression_mask = torch.tensor(tokenized_turn["content_compression_mask"])
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜><｜User｜>Problem: The quadratic $x^2+(2.6)x+3.6$ can be written in the form $(x+b)^2+c$, where $b$ and $c$ are constants. What is $b+c$ (as a decimal)?

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>

Okay, so I have this quadratic equation: \( x^2 + 2.6x + 3.6 \). The problem says I need to write it in the form \( (x + b)^2 + c \), and then find the sum of \( b \) and \( c \). Hmm, I think this is about completing the square. Let me recall how that works.

Completing the square is a method used to convert a quadratic equation from standard form to vertex form, which is exactly what they're asking for here. The standard form is \( ax^2 + bx + c \), and the vertex form is \( a(x - h)^2 + k \). In this case, the quadratic is already in standard form, so I need to manipulate it to get it into the vertex form.

The given quadratic is \( x^2 + 2.6x + 3.6 \). Since the coefficient of \( x^2 \) is 1, that should make things a bit

In [30]:
labels = torch.cat([question_input_ids.cuda(), generated_ids_new.cuda()], dim=1)
print(tokenizer.decode(labels[-1]))

question_content_mask = content_compression_mask[: int(input_part_end) + 1].clone()
question_content_mask[question_content_mask == 0] = 4
question_content_mask[question_content_mask == 1] = 0
question_content_mask[question_content_mask == 4] = 1
train_content_mask_new = torch.cat(
    [
        question_content_mask,
        torch.ones(
            generated_ids_new.shape[1],
        ),
    ]
).long()
print(question_content_mask)

with torch.no_grad():
    generated_embeds = model.get_input_embeddings()(generated_ids_new)
    new_input_embeds = torch.cat(
        [
            input_ids_embeds,
            generated_embeds,
        ],
        dim=1,
    )
labels[:, train_content_mask_new == 0] = -100
# new_input_embeds
labels

<｜begin▁of▁sentence｜><｜User｜>Problem: The quadratic $x^2+(2.6)x+3.6$ can be written in the form $(x+b)^2+c$, where $b$ and $c$ are constants. What is $b+c$ (as a decimal)?

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>
Okay, so I have this quadratic equation: \( x^2 + 2.6x + 3.6 \). The problem says I need to write it in the form \( (x + b)^2 + c \), and then find the sum of \( b \) and \( c \). Hmm, I think this is about completing the square. Let me recall how that works.

Completing the square is a method used to convert a quadratic equation from standard form to vertex form, which is exactly what they're asking for here. The standard form is \( ax^2 + bx + c \), and the vertex form is \( a(x - h)^2 + k \). In this case, the quadratic is already in standard form, so I need to manipulate it to get it into the vertex form.

The given quadratic is \( x^2 + 2.6x + 3.6 \). Since the coefficient of \( x^2 \) is 1, that should make things a bit 

tensor([[151646, 151644,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,   -100,
           -100,   -100,   -100,   -100,   -100,   -100,   -100, 151645, 151648,
            198,  32313,     11,    773,    358,    614,    419,  79151,  23606,
             25,  17767,    856,     61,     17,    488,    220,     17,     13,
             21,     87,    488,    220,     18,     13,     21,   1124,    568,
            576,   3491,   2727,    358,   1184,    311,   3270,    432,    304,
            279,   1352,  17

In [37]:
eos_id = tokenizer.eos_token_id
bos_id = tokenizer.bos_token_id
end_think_id = torch.tensor(tokenizer.encode("</think>", add_special_tokens=False)[0])
start_think_id = torch.tensor(
    tokenizer.encode("<｜Assistant｜><think>\n", add_special_tokens=False)
)
user_id = torch.tensor(tokenizer.encode("<｜User｜>", add_special_tokens=False)[0])

# train_attention_mask = (
#     input_ids == bos_id | input_ids == start_think_id | input_ids == user_id
# )

In [31]:
with torch.no_grad():
    original_loss = model(
        inputs_embeds=new_input_embeds,
        # attention_mask=train_attention_mask.unsqueeze(0).cuda(),
        # attention_mask=torch.ones(new_input_embeds.size()[:2]).cuda().long(),
        labels=labels,
    ).loss
original_loss

tensor(0.2947, device='cuda:0')

In [85]:
# compression_tokens = 1
# compression_tokens = max_new_tokens // 2
compression_tokens = max_new_tokens // 8
compression_tensor = torch.nn.Parameter(
    torch.rand_like(
        new_input_embeds[:, :compression_tokens, :],
    )
    * model.get_input_embeddings().weight.data.std(),
    requires_grad=True,
)
compressed_inputs_embeds = torch.cat(
    [
        input_ids_embeds.detach(),
        compression_tensor,
        generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
    ],
    dim=1,
)

In [64]:
compression_tensor.shape, compression_tokens

(torch.Size([1, 50, 1536]), 50)

In [98]:
input_ids_embeds

tensor([[[-0.0275,  0.0029, -0.0052,  ...,  0.0306, -0.0159,  0.0216],
         [-0.0344,  0.0019, -0.0027,  ...,  0.0216, -0.0178,  0.0244],
         [-0.0068, -0.0247, -0.0413,  ..., -0.0012,  0.0081,  0.0493],
         ...,
         [-0.0242,  0.0099,  0.0051,  ...,  0.0217, -0.0115,  0.0197],
         [-0.0203,  0.0073, -0.0091,  ...,  0.0225, -0.0061,  0.0275],
         [ 0.0398, -0.0708,  0.0342,  ..., -0.0579,  0.0427, -0.0625]]],
       device='cuda:0')

In [91]:
compression_tensor

Parameter containing:
tensor([[[0.0253, 0.0171, 0.0290,  ..., 0.0164, 0.0102, 0.0168],
         [0.0051, 0.0215, 0.0075,  ..., 0.0093, 0.0031, 0.0203],
         [0.0280, 0.0290, 0.0146,  ..., 0.0178, 0.0261, 0.0190],
         ...,
         [0.0139, 0.0011, 0.0014,  ..., 0.0242, 0.0159, 0.0255],
         [0.0244, 0.0278, 0.0026,  ..., 0.0048, 0.0257, 0.0191],
         [0.0091, 0.0152, 0.0013,  ..., 0.0037, 0.0016, 0.0016]]],
       device='cuda:0', requires_grad=True)

In [96]:
compression_tensor

Parameter containing:
tensor([[[ 0.0274,  0.0208,  0.0213,  ...,  0.0144,  0.0100,  0.0129],
         [-0.0014,  0.0313, -0.0052,  ...,  0.0343,  0.0071,  0.0129],
         [ 0.0346,  0.0378,  0.0188,  ...,  0.0149,  0.0327,  0.0096],
         ...,
         [ 0.0293, -0.0169, -0.0487,  ...,  0.0626,  0.0307,  0.0178],
         [ 0.0277,  0.0375, -0.0113,  ...,  0.0166,  0.0140, -0.0069],
         [ 0.0301,  0.0065, -0.0088,  ..., -0.0039,  0.0399, -0.0068]]],
       device='cuda:0', requires_grad=True)

In [97]:
input_ids_embeds

tensor([[[-0.0275,  0.0029, -0.0052,  ...,  0.0306, -0.0159,  0.0216],
         [-0.0344,  0.0019, -0.0027,  ...,  0.0216, -0.0178,  0.0244],
         [-0.0068, -0.0247, -0.0413,  ..., -0.0012,  0.0081,  0.0493],
         ...,
         [-0.0242,  0.0099,  0.0051,  ...,  0.0217, -0.0115,  0.0197],
         [-0.0203,  0.0073, -0.0091,  ...,  0.0225, -0.0061,  0.0275],
         [ 0.0398, -0.0708,  0.0342,  ..., -0.0579,  0.0427, -0.0625]]],
       device='cuda:0')

In [88]:
generated_embeds[:, -(max_new_tokens // 2) :, :]

tensor([[[ 0.0505,  0.0034, -0.0226,  ..., -0.0679,  0.0032,  0.0082],
         [ 0.0483, -0.0315,  0.0374,  ..., -0.0537,  0.0205, -0.0469],
         [ 0.0110,  0.0070,  0.0408,  ..., -0.0236, -0.0127, -0.0437],
         ...,
         [ 0.0215, -0.0376,  0.0186,  ...,  0.0107,  0.0139, -0.0659],
         [ 0.0432,  0.0038,  0.0366,  ..., -0.0649,  0.0442,  0.0317],
         [ 0.0087, -0.0172,  0.0258,  ..., -0.0123,  0.0122, -0.0039]]],
       device='cuda:0')

In [89]:
generated_embeds[:, -(max_new_tokens // 2) :, :]

tensor([[[ 0.0505,  0.0034, -0.0226,  ..., -0.0679,  0.0032,  0.0082],
         [ 0.0483, -0.0315,  0.0374,  ..., -0.0537,  0.0205, -0.0469],
         [ 0.0110,  0.0070,  0.0408,  ..., -0.0236, -0.0127, -0.0437],
         ...,
         [ 0.0215, -0.0376,  0.0186,  ...,  0.0107,  0.0139, -0.0659],
         [ 0.0432,  0.0038,  0.0366,  ..., -0.0649,  0.0442,  0.0317],
         [ 0.0087, -0.0172,  0.0258,  ..., -0.0123,  0.0122, -0.0039]]],
       device='cuda:0')

In [93]:
# compressed_labels = question_input_ids
# comp
question_labels = question_input_ids.clone()
question_labels[0][question_content_mask == 0] = -100
question_labels = question_labels.cuda()
compressed_part = torch.ones(compression_tensor.shape[:2]).long().cuda()

compressed_labels = torch.cat(
    [
        question_labels,
        compressed_part,
        generated_ids_new[:, -(max_new_tokens // 2) :],
    ],
    dim=-1,
)
# compressed_labels

In [94]:
with torch.no_grad():
    compression_loss = model(
        inputs_embeds=compressed_inputs_embeds,
        labels=compressed_labels,
    ).loss

compression_loss

tensor(2.1431, device='cuda:0')

In [18]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Requires Gradient: {param.requires_grad}")

In [126]:
compression_loss.item() <= original_loss.item()

False

In [95]:
epoch_amount = 100
# epoch_amount = 500

optimizer = torch.optim.AdamW([compression_tensor], lr=0.001)
for epoch in range(epoch_amount):
    print(epoch)
    compressed_inputs_embeds = torch.cat(
        [
            input_ids_embeds.detach(),
            compression_tensor,
            generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
        ],
        dim=1,
    )
    compression_loss = model(
        inputs_embeds=compressed_inputs_embeds,
        labels=compressed_labels,
    ).loss
    print(compression_loss)
    compression_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if compression_loss.item() <= original_loss.item():
        break

0
tensor(2.1431, device='cuda:0', grad_fn=<NllLossBackward0>)
1
tensor(1.5753, device='cuda:0', grad_fn=<NllLossBackward0>)
2
tensor(1.4054, device='cuda:0', grad_fn=<NllLossBackward0>)
3
tensor(1.1314, device='cuda:0', grad_fn=<NllLossBackward0>)
4
tensor(0.9445, device='cuda:0', grad_fn=<NllLossBackward0>)
5
tensor(0.8756, device='cuda:0', grad_fn=<NllLossBackward0>)
6
tensor(0.8547, device='cuda:0', grad_fn=<NllLossBackward0>)
7
tensor(0.8284, device='cuda:0', grad_fn=<NllLossBackward0>)
8
tensor(0.7996, device='cuda:0', grad_fn=<NllLossBackward0>)
9
tensor(0.7731, device='cuda:0', grad_fn=<NllLossBackward0>)
10
tensor(0.7524, device='cuda:0', grad_fn=<NllLossBackward0>)
11
tensor(0.7396, device='cuda:0', grad_fn=<NllLossBackward0>)
12
tensor(0.7249, device='cuda:0', grad_fn=<NllLossBackward0>)
13
tensor(0.7114, device='cuda:0', grad_fn=<NllLossBackward0>)
14
tensor(0.6989, device='cuda:0', grad_fn=<NllLossBackward0>)
15
tensor(0.6880, device='cuda:0', grad_fn=<NllLossBackward0>)
16

In [55]:
compression_tensor

Parameter containing:
tensor([[[ 0.1333, -0.0525,  0.0161,  ...,  0.0050,  0.0536,  0.0615],
         [ 0.0280, -0.0860, -0.0381,  ..., -0.0432,  0.0546, -0.0791],
         [-0.0107,  0.0577, -0.0345,  ..., -0.0124,  0.1097,  0.0769],
         ...,
         [ 0.1454,  0.0869,  0.1768,  ...,  0.0386,  0.0551, -0.0129],
         [ 0.1206, -0.0238,  0.0889,  ...,  0.0130,  0.0191,  0.0909],
         [ 0.1462,  0.2155, -0.0516,  ...,  0.1534,  0.2087,  0.1910]]],
       device='cuda:0', requires_grad=True)

In [149]:
new_input_embeds.shape[1]

139

In [60]:
compression_loss #/ compressed_inputs_embeds.shape[1]

tensor(0.5967, device='cuda:0', grad_fn=<NllLossBackward0>)

In [75]:
original_loss #/ new_input_embeds.shape[1]

tensor(0.2947, device='cuda:0')

In [72]:
with torch.no_grad():

    compressed_inputs_embeds = torch.cat(
        [
            input_ids_embeds.detach(),
            # compression_tensor,
            torch.rand_like(compression_tensor).cuda(),
            # torch.zeros_like(compression_tensor).cuda(),
            generated_embeds[:, -(max_new_tokens // 2) :, :].detach(),
        ],
        dim=1,
    )
    generated_ids_compressed = model.generate(
        inputs_embeds=compressed_inputs_embeds,
        attention_mask=torch.ones(
            compressed_inputs_embeds.shape[:2],
            device="cuda",
        ).long(),
        max_new_tokens=4096,
        # max_new_tokens=5,
        do_sample=False,
        # do_sample=True,
        # temperature=0.6,
        # top_p=0.95,
    )
    # break
generated_result = tokenizer.decode(generated_ids_compressed[-1])
print(generated_result)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


 that, it becomes:

\( x^2 + 2bx + b^2 + c \)

Now, I can compare this with the original quadratic:

Original: \( x^2 + 2.6x + 3.6 \)

Expanded vertex form: \( x^2 + 2bx + (b^2 + c) \)

So, by comparing coefficients, I can set up equations:

1. Coefficient of \( x \): \( 2b = 2.6 \)
2. Constant term: \( b^2 + c = 3.6 \)

From the first equation, I can solve for \( b \):

\( 2b = 2.6 \)

Divide both sides by 2:

\( b = 2.6 / 2 \)

Calculating that:

\( 2.6 divided by 2 is 1.3 \). So, \( b = 1.3 \)

Now, plug this value of \( b \) into the second equation to find \( c \):

\( (1.3)^2 + c = 3.6 \)

Calculate \( (1.3)^2 \):

\( 1.3 * 1.3 = 1.69 \)

So, the equation becomes:

\( 1.69 + c = 3.6 \)

Subtract 1.69 from both sides:

\( c = 3.6 - 1.69 \)

Calculating that:

\( 3.6 minus 1.69 is 1.91 \). So, \( c = 1.91 \)

Now, the problem asks for \( b + c \). So, adding them together:

\( b + c = 1.3 + 1.91 \)

Adding those:

\( 1.3 + 1.91 = 3.21 \)

So, \( b + c = 3.21 \)

Wait, let me double

In [73]:
# print()
gold_answer = correct_dataset[dataset_pos]["answer"]
answer = dataset_answer_filter(gold_answer)
model_answer = model_answer_filter(generated_result)
if is_equiv(answer, model_answer):
    correct_items += 1
    print("CORRECT")
else:
    print("WRONG", gold_answer)
    print(generated_result)
compressed_total_len = generated_ids_compressed.shape[1]
print(
    generated_ids_compressed.shape[1],
    compressed_total_len,
    # torch.tensor(generated_ids).shape[1],
)

CORRECT
1111 1111


In [74]:
(
    compression_tensor.shape[1]
    + generated_embeds[:, -(max_new_tokens // 2) :, :].shape[1]
    + generated_ids_compressed.shape[1]
)

1361

In [71]:
len(tokenizer.encode(
    correct_dataset[dataset_pos]["model_answer"],
    add_special_tokens=False,
))

1639